In [2]:
import pandas as pd
import numpy as np

na_cat_preprocess_path = r'C:\Users\OJH\OneDrive\SKKU\연구자료\Catalyst AI\Database\Na_Cat_preprocess.csv'
na_w_mn_cat_path = r'C:\Users\OJH\OneDrive\SKKU\연구자료\Catalyst AI\New Database\Na_W_Mn_Cat.csv'

na_cat_preprocess_data = pd.read_csv(na_cat_preprocess_path)
na_w_mn_cat_data = pd.read_csv(na_w_mn_cat_path)

na_cat_preprocess_data.head(), na_w_mn_cat_data.head()

(   Promotor  Preparation  Temperature, K  p(CH4)/p(O2)  p total, bar  \
 0         0            1            1017           8.0           1.0   
 1         0            1            1021           8.0           1.0   
 2         0            1            1023           8.1           1.0   
 3         0            1            1018           8.1           1.0   
 4         0            1            1026           8.0           1.0   
 
    Contact time, s  X(O2), %  X(CH4), %  S(COx), %  S(C2=), %  ...  W_C_mol%  \
 0             0.19      39.0        3.0        0.0       23.0  ...       0.0   
 1             0.16      98.0       16.0        0.0       24.0  ...       0.0   
 2            38.40      97.0       15.0        0.0       25.0  ...       0.0   
 3             0.25      92.0       16.0        0.0       26.0  ...       0.0   
 4             0.47      95.0       10.0        0.0       16.0  ...       0.0   
 
    Ti_C_mol%  Ce_C_mol%  S_A_mol%  P_A_mol%  Cl_A_mol%  Si_S_mol%  C_S_

In [3]:
def rename_columns(data):
    new_columns = []
    for col in data.columns:
        if "Cation" in col and "mol%" in col:
            cation_value = data[col.replace(" mol%", "")].iloc[0]  # Getting the corresponding cation name for the first row
            new_columns.append(f"{cation_value}_C_mol%")
        elif "Anion" in col and "mol%" in col:
            anion_value = data[col.replace(" mol%", "")].iloc[0]
            new_columns.append(f"{anion_value}_A_mol%")
        elif "Support" in col and "mol%" in col:
            support_value = data[col.replace(" mol%", "")].iloc[0]
            new_columns.append(f"{support_value}_S_mol%")
        else:
            new_columns.append(col)
    return new_columns

In [4]:
# Renaming columns
na_w_mn_cat_data.columns = rename_columns(na_w_mn_cat_data)

# Step 3: Modify the 'Preparation' column
# Assuming 'Preparation' is a column in 'Na_W_Mn_Cat.csv', converting 'n.a.' to 0 and others to 1
na_w_mn_cat_data['Preparation'] = na_w_mn_cat_data['Preparation'].apply(lambda x: 0 if x == 'n.a.' else 1)

# Step 4: Remove specified columns
cols_to_remove = ['X(O2)', 'S(COx)', 'S(C2=)', 'S(C2-)']
na_w_mn_cat_data = na_w_mn_cat_data.drop(columns=[col for col in cols_to_remove if col in na_w_mn_cat_data.columns], errors='ignore')

# Displaying the first few rows of the transformed data
na_w_mn_cat_data.head()

,Cation 1,W_C_mol%,Cation 2,Na_C_mol%,Cation 3,Mn_C_mol%,Cation 4,nan_C_mol%,Cation 5,nan_C_mol%,...,Preparation,Temperature,p(CH4),p(O2),p(CH4)/p(O2),p total,Contact time,X(CH4),S(C2),Y(C2)
0,W,0.5,Na,33.20,Mn,66.30,NaN,NaN,NaN,NaN,...,0,1089,0.67,0.33,2.0,1.0,3.00,41.0,44.0,18.0
1,W,2.3,Na,19.50,Mn,58.70,NaN,NaN,NaN,NaN,...,1,1073,0.67,0.33,2.0,1.0,3.00,40.7,43.5,17.7
2,Na,8.9,W,4.50,Mn,3.50,NaN,NaN,NaN,NaN,...,1,973,0.40,0.10,4.0,1.0,1.33,40.0,63.0,25.2
3,Na,2.1,W,1.05,Mn,2.25,NaN,NaN,NaN,NaN,...,1,1092,0.68,0.32,2.1,1.0,0.15,37.0,48.0,17.8
4,Na,2.1,W,1.05,Mn,2.25,NaN,NaN,NaN,NaN,...,1,1124,0.69,0.31,2.2,1.0,0.15,40.0,45.0,18.0


In [5]:
na_w_mn_cat_data.columns

Index(['Cation 1', 'W_C_mol%', 'Cation 2', 'Na_C_mol%', 'Cation 3',
       'Mn_C_mol%', 'Cation 4', 'nan_C_mol%', 'Cation 5', 'nan_C_mol%',
       'Cation 6', 'nan_C_mol%', 'Anion 1', 'nan_A_mol%', 'Anion 2',
       'nan_A_mol%', 'Support 1', 'nan_S_mol%', 'Support 2', 'nan_S_mol%',
       'Support 3', 'nan_S_mol%', 'Preparation', 'Temperature', 'p(CH4)',
       'p(O2)', 'p(CH4)/p(O2)', 'p total', 'Contact time', 'X(CH4)', 'S(C2)',
       'Y(C2)'],
      dtype='object')

In [6]:
def update_mol_percentages(row, unique_cations, unique_anions, unique_supports):
    # Initialize all possible cation, anion, and support mol% to 0
    updated_data = {f'{cation}_C_mol%': 0 for cation in unique_cations}
    updated_data.update({f'{anion}_A_mol%': 0 for anion in unique_anions})
    updated_data.update({f'{support}_S_mol%': 0 for support in unique_supports})

    # Update the mol% for present cations, anions, and supports in the row
    for i in range(1, 7):  # Assuming there are up to 6 cations
        cation = row.get(f'Cation {i}')
        cation_mol = row.get(f'Cation {i} mol%')
        if pd.notna(cation):
            updated_data[f'{cation}_C_mol%'] = cation_mol

    for i in range(1, 3):  # Assuming there are up to 2 anions
        anion = row.get(f'Anion {i}')
        anion_mol = row.get(f'Anion {i} mol%')
        if pd.notna(anion):
            updated_data[f'{anion}_A_mol%'] = anion_mol

    for i in range(1, 4):  # Assuming there are up to 3 supports
        support = row.get(f'Support {i}')
        support_mol = row.get(f'Support {i} mol%')
        if pd.notna(support):
            updated_data[f'{support}_S_mol%'] = support_mol

    return updated_data

# Identify all unique cations, anions, and supports from the dataset
unique_cations = set(na_w_mn_cat_data[[f'Cation {i}' for i in range(1,7)]].stack().dropna())
unique_anions = set(na_w_mn_cat_data[[f'Anion {i}' for i in range(1,3)]].stack().dropna())
unique_supports = set(na_w_mn_cat_data[[f'Support {i}' for i in range(1,4)]].stack().dropna())

# Update each row in the dataframe
new_columns_data = [update_mol_percentages(row, unique_cations, unique_anions, unique_supports) for index, row in na_w_mn_cat_data.iterrows()]

# Convert the list of dictionaries to a DataFrame
updated_df = pd.DataFrame(new_columns_data)

# Show the first few rows of the updated DataFrame
updated_df.head()

,Mg_C_mol%,Nb_C_mol%,Fe_C_mol%,K_C_mol%,Na_C_mol%,Ce_C_mol%,Rh_C_mol%,Ag_C_mol%,La_C_mol%,Ga_C_mol%,...,Si_S_mol%,Ca_S_mol%,Al_S_mol%,La_S_mol%,Mg_S_mol%,C_S_mol%,Ti_S_mol%,Fe_S_mol%,Zr_S_mol%,Sr_S_mol%
0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
updated_df.columns

Index(['Mg_C_mol%', 'Nb_C_mol%', 'Fe_C_mol%', 'K_C_mol%', 'Na_C_mol%',
       'Ce_C_mol%', 'Rh_C_mol%', 'Ag_C_mol%', 'La_C_mol%', 'Ga_C_mol%',
       'Cs_C_mol%', ' Na _C_mol%', 'Pt_C_mol%', 'Mn_C_mol%', 'W_C_mol%',
       'Zr_C_mol%', 'Na _C_mol%', 'Li_C_mol%', 'Ti_C_mol%', 'Ir_C_mol%',
       'Sn_C_mol%', 'Ge_C_mol%', 'P_A_mol%', 'S_A_mol%', 'Cl_A_mol%',
       'Si_S_mol%', 'Ca_S_mol%', 'Al_S_mol%', 'La_S_mol%', 'Mg_S_mol%',
       'C_S_mol%', 'Ti_S_mol%', 'Fe_S_mol%', 'Zr_S_mol%', 'Sr_S_mol%'],
      dtype='object')

In [8]:
# Let's start fresh with the following steps based on the instructions:

# Load the data from the original file
na_w_mn_cat_data = pd.read_csv(na_w_mn_cat_path)

# Step 1: Collect all unique cations, anions, and supports names from the original dataset
unique_cations = set(na_w_mn_cat_data[[f'Cation {i}' for i in range(1, 7)]].stack().dropna())
unique_anions = set(na_w_mn_cat_data[[f'Anion {i}' for i in range(1, 3)]].stack().dropna())
unique_supports = set(na_w_mn_cat_data[[f'Support {i}' for i in range(1, 4)]].stack().dropna())

# Step 2: Define a function to update mol% values for each row in the dataset
def update_mol_percentages(row):
    # Initialize columns for cations, anions, and supports with 0
    updated_row = {f"{cation}_C_mol%": 0 for cation in unique_cations}
    updated_row.update({f"{anion}_A_mol%": 0 for anion in unique_anions})
    updated_row.update({f"{support}_S_mol%": 0 for support in unique_supports})
    
    # Update values from the row
    for i in range(1, 7):  # Assuming up to 6 cations
        cation = row.get(f'Cation {i}')
        cation_mol = row.get(f'Cation {i} mol%')
        if pd.notna(cation) and pd.notna(cation_mol):
            updated_row[f"{cation}_C_mol%"] = cation_mol
            
    for i in range(1, 3):  # Assuming up to 2 anions
        anion = row.get(f'Anion {i}')
        anion_mol = row.get(f'Anion {i} mol%')
        if pd.notna(anion) and pd.notna(anion_mol):
            updated_row[f"{anion}_A_mol%"] = anion_mol
            
    for i in range(1, 4):  # Assuming up to 3 supports
        support = row.get(f'Support {i}')
        support_mol = row.get(f'Support {i} mol%')
        if pd.notna(support) and pd.notna(support_mol):
            updated_row[f"{support}_S_mol%"] = support_mol
            
    return updated_row

# Step 3: Apply the function to each row in the original dataframe
updated_rows = [update_mol_percentages(row) for index, row in na_w_mn_cat_data.iterrows()]

# Step 4: Convert the updated rows back into a dataframe
updated_mol_df = pd.DataFrame(updated_rows)

# Step 5: Keep the other specified columns
other_columns = ['Preparation', 'Temperature', 'p(CH4)', 'p(O2)', 'p(CH4)/p(O2)', 'p total', 'Contact time', 'X(CH4)', 'S(C2)', 'Y(C2)']
remaining_data = na_w_mn_cat_data[other_columns]

# Step 6: Concatenate the updated mol% data with the remaining columns
final_df = pd.concat([updated_mol_df, remaining_data], axis=1)

# Show the first few rows of the final updated DataFrame
final_df.head()

,Mg_C_mol%,Nb_C_mol%,Fe_C_mol%,K_C_mol%,Na_C_mol%,Ce_C_mol%,Rh_C_mol%,Ag_C_mol%,La_C_mol%,Ga_C_mol%,...,Preparation,Temperature,p(CH4),p(O2),p(CH4)/p(O2),p total,Contact time,X(CH4),S(C2),Y(C2)
0,0.0,0.0,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,...,n.a.,1089,0.67,0.33,2.0,1.0,3.00,41.0,44.0,18.0
1,0.0,0.0,0.0,0.0,19.5,0.0,0.0,0.0,0.0,0.0,...,Impregnation,1073,0.67,0.33,2.0,1.0,3.00,40.7,43.5,17.7
2,0.0,0.0,0.0,0.0,8.9,0.0,0.0,0.0,0.0,0.0,...,Therm.decomp.,973,0.40,0.10,4.0,1.0,1.33,40.0,63.0,25.2
3,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,...,Solid-phase technique,1092,0.68,0.32,2.1,1.0,0.15,37.0,48.0,17.8
4,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,...,Solid-phase technique,1124,0.69,0.31,2.2,1.0,0.15,40.0,45.0,18.0


In [9]:
# Assuming 'Preparation' is a column in 'Na_W_Mn_Cat.csv', converting 'n.a.' to 0 and others to 1
final_df['Preparation'] = final_df['Preparation'].apply(lambda x: 0 if x == 'n.a.' else 1)

In [10]:
final_df

,Mg_C_mol%,Nb_C_mol%,Fe_C_mol%,K_C_mol%,Na_C_mol%,Ce_C_mol%,Rh_C_mol%,Ag_C_mol%,La_C_mol%,Ga_C_mol%,...,Preparation,Temperature,p(CH4),p(O2),p(CH4)/p(O2),p total,Contact time,X(CH4),S(C2),Y(C2)
0,0.0,0.0,0.0,0.0,33.20,0.0,0.0,0.0,0.0,0.0,...,0,1089,0.67,0.33,2.0,1.0,3.00,41.0,44.0,18.0
1,0.0,0.0,0.0,0.0,19.50,0.0,0.0,0.0,0.0,0.0,...,1,1073,0.67,0.33,2.0,1.0,3.00,40.7,43.5,17.7
2,0.0,0.0,0.0,0.0,8.90,0.0,0.0,0.0,0.0,0.0,...,1,973,0.40,0.10,4.0,1.0,1.33,40.0,63.0,25.2
3,0.0,0.0,0.0,0.0,2.10,0.0,0.0,0.0,0.0,0.0,...,1,1092,0.68,0.32,2.1,1.0,0.15,37.0,48.0,17.8
4,0.0,0.0,0.0,0.0,2.10,0.0,0.0,0.0,0.0,0.0,...,1,1124,0.69,0.31,2.2,1.0,0.15,40.0,45.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,0.0,0.0,0.0,0.0,4.39,0.0,0.0,0.0,0.0,0.0,...,1,953,0.50,0.10,5.0,1.0,0.16,14.0,52.4,7.4
867,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,1,973,0.38,0.13,3.0,1.0,0.32,28.1,53.1,14.9
868,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,1,973,0.38,0.13,3.0,1.0,0.53,33.8,55.0,18.6
869,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,1,973,0.38,0.13,3.0,1.0,0.79,32.9,54.8,18.1


In [12]:
final_df_to_csv_path = r'C:\Users\OJH\OneDrive\SKKU\연구자료\Catalyst AI\New Database\Na_W_Mn_Cat_preprocess.csv'
final_df.to_csv(final_df_to_csv_path, index=False)

In [17]:
data = pd.read_csv(r'C:\Users\OJH\OneDrive\SKKU\연구자료\Catalyst AI\New Database\Na_W_Mn_Cat_preprocess.csv')
data.head()

,Mg_C_mol%,Nb_C_mol%,Fe_C_mol%,K_C_mol%,Na_C_mol%,Ce_C_mol%,Rh_C_mol%,Ag_C_mol%,La_C_mol%,Ga_C_mol%,...,Preparation,Temperature,p(CH4),p(O2),p(CH4)/p(O2),p total,Contact time,X(CH4),S(C2),Y(C2)
0,0.0,0.0,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,...,0,1089,0.67,0.33,2.0,1.0,3.00,41.0,44.0,18.0
1,0.0,0.0,0.0,0.0,19.5,0.0,0.0,0.0,0.0,0.0,...,1,1073,0.67,0.33,2.0,1.0,3.00,40.7,43.5,17.7
2,0.0,0.0,0.0,0.0,8.9,0.0,0.0,0.0,0.0,0.0,...,1,973,0.40,0.10,4.0,1.0,1.33,40.0,63.0,25.2
3,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,...,1,1092,0.68,0.32,2.1,1.0,0.15,37.0,48.0,17.8
4,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,...,1,1124,0.69,0.31,2.2,1.0,0.15,40.0,45.0,18.0


In [18]:
# Define a function to fill missing 'Y(C2)' values based on 'X(CH4)' and 'S(C2)'
def fill_missing_Y_C2(row):
    # If 'Y(C2)' is missing but both 'X(CH4)' and 'S(C2)' are available
    if pd.isna(row['Y(C2)']) and pd.notna(row['X(CH4)']) and pd.notna(row['S(C2)']):
        return row['X(CH4)'] * row['S(C2)'] / 100
    else:
        return row['Y(C2)']

# Apply the function to the 'Y(C2)' column
data['Y(C2)'] = data.apply(fill_missing_Y_C2, axis=1)

# Display the first few rows of the updated DataFrame to verify changes
data.head()

,Mg_C_mol%,Nb_C_mol%,Fe_C_mol%,K_C_mol%,Na_C_mol%,Ce_C_mol%,Rh_C_mol%,Ag_C_mol%,La_C_mol%,Ga_C_mol%,...,Preparation,Temperature,p(CH4),p(O2),p(CH4)/p(O2),p total,Contact time,X(CH4),S(C2),Y(C2)
0,0.0,0.0,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,...,0,1089,0.67,0.33,2.0,1.0,3.00,41.0,44.0,18.0
1,0.0,0.0,0.0,0.0,19.5,0.0,0.0,0.0,0.0,0.0,...,1,1073,0.67,0.33,2.0,1.0,3.00,40.7,43.5,17.7
2,0.0,0.0,0.0,0.0,8.9,0.0,0.0,0.0,0.0,0.0,...,1,973,0.40,0.10,4.0,1.0,1.33,40.0,63.0,25.2
3,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,...,1,1092,0.68,0.32,2.1,1.0,0.15,37.0,48.0,17.8
4,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,...,1,1124,0.69,0.31,2.2,1.0,0.15,40.0,45.0,18.0


In [20]:
data.drop(columns=['p(CH4)','p(O2)','X(CH4)', 'S(C2)'], axis=1, inplace=True)

In [21]:
data

,Mg_C_mol%,Nb_C_mol%,Fe_C_mol%,K_C_mol%,Na_C_mol%,Ce_C_mol%,Rh_C_mol%,Ag_C_mol%,La_C_mol%,Ga_C_mol%,...,Ti_S_mol%,Fe_S_mol%,Zr_S_mol%,Sr_S_mol%,Preparation,Temperature,p(CH4)/p(O2),p total,Contact time,Y(C2)
0,0.0,0.0,0.0,0.0,33.20,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,1089,2.0,1.0,3.00,18.0
1,0.0,0.0,0.0,0.0,19.50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1073,2.0,1.0,3.00,17.7
2,0.0,0.0,0.0,0.0,8.90,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,973,4.0,1.0,1.33,25.2
3,0.0,0.0,0.0,0.0,2.10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1092,2.1,1.0,0.15,17.8
4,0.0,0.0,0.0,0.0,2.10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1124,2.2,1.0,0.15,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,0.0,0.0,0.0,0.0,4.39,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,953,5.0,1.0,0.16,7.4
867,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,973,3.0,1.0,0.32,14.9
868,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,973,3.0,1.0,0.53,18.6
869,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,973,3.0,1.0,0.79,18.1


In [23]:
data = data.dropna(subset=['Contact time', 'Y(C2)'])

data

,Mg_C_mol%,Nb_C_mol%,Fe_C_mol%,K_C_mol%,Na_C_mol%,Ce_C_mol%,Rh_C_mol%,Ag_C_mol%,La_C_mol%,Ga_C_mol%,...,Ti_S_mol%,Fe_S_mol%,Zr_S_mol%,Sr_S_mol%,Preparation,Temperature,p(CH4)/p(O2),p total,Contact time,Y(C2)
0,0.0,0.0,0.0,0.0,33.20,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,1089,2.0,1.0,3.00,18.0
1,0.0,0.0,0.0,0.0,19.50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1073,2.0,1.0,3.00,17.7
2,0.0,0.0,0.0,0.0,8.90,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,973,4.0,1.0,1.33,25.2
3,0.0,0.0,0.0,0.0,2.10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1092,2.1,1.0,0.15,17.8
4,0.0,0.0,0.0,0.0,2.10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1124,2.2,1.0,0.15,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,0.0,0.0,0.0,0.0,4.39,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,953,5.0,1.0,0.16,7.4
867,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,973,3.0,1.0,0.32,14.9
868,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,973,3.0,1.0,0.53,18.6
869,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,973,3.0,1.0,0.79,18.1


In [24]:
data_path = r'C:\Users\OJH\OneDrive\SKKU\연구자료\Catalyst AI\New Database\Na_W_Mn_Cat_preprocess_rev.csv'
data.to_csv(data_path, index=False)